In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.templating import Jinja2Templates
from ultralytics import YOLO
import numpy as np
import cv2
import base64
import threading

app = FastAPI()
templates = Jinja2Templates(directory="templates")

# ===== LOAD YOLOv8 =====
model = YOLO("yolov8n.pt")  # nano = fastest

LCD_CLASSES = {"tv", "monitor"}

@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


@app.post("/detect")
async def detect(payload: dict):
    image_data = payload["image"].split(",")[1]
    img_bytes = base64.b64decode(image_data)

    np_arr = np.frombuffer(img_bytes, np.uint8)
    frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    results = model(frame, conf=0.5, verbose=False)

    lcd_detected = False

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]

            if label in LCD_CLASSES:
                lcd_detected = True

    return JSONResponse({"lcd_detected": lcd_detected})


RuntimeError: asyncio.run() cannot be called from a running event loop

In [8]:
from flask import Flask, render_template, request, jsonify
from ultralytics import YOLO
import numpy as np
import cv2
import base64

app = Flask(__name__, template_folder="templates")

# ===== LOAD YOLOv8 =====
model = YOLO("yolov8n.pt")  # nano = fastest

LCD_CLASSES = {"tv", "monitor"}

# ================= HOME =================
@app.route("/", methods=["GET"])
def home():
    return render_template("index.html")

# ================= DETECTION =================
@app.route("/detect", methods=["POST"])
def detect():
    payload = request.get_json()

    image_data = payload["image"].split(",")[1]
    img_bytes = base64.b64decode(image_data)

    np_arr = np.frombuffer(img_bytes, np.uint8)
    frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

    results = model(frame, conf=0.5, verbose=False)

    lcd_detected = False
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]
            if label in LCD_CLASSES:
                lcd_detected = True
                break

    return jsonify({"lcd_detected": lcd_detected})

# ================== RUN ==================
import threading
def run_app():
    app.run(
        host="0.0.0.0",
        port=5000,
        debug=False,
        use_reloader=False
    )

threading.Thread(target=run_app, daemon=True).start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.87.11.250:5000
Press CTRL+C to quit


In [1]:
import os
from fastapi.templating import Jinja2Templates  
templates = Jinja2Templates(directory="templates")
print("Template directory exists:", os.path.exists("templates"))

Template directory exists: True


In [ ]:
def run_app():
    app.run(
        host="0.0.0.0",
        port=5000,
        debug=False,
        use_reloader=False
    )

threading.Thread(target=run_app, daemon=True).start()

RuntimeError: asyncio.run() cannot be called from a running event loop